In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import time
import re

from sklearn.metrics import classification_report

from bs4 import BeautifulSoup             


from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Tools for creating ngrams and vectorizing input data
from gensim.models import Word2Vec, Phrases

# Configs
pd.options.display.float_format = '{:,.4f}'.format
sns.set(style="whitegrid")
plt.style.use('seaborn')
seed = 42
np.random.seed(seed)

C:\Users\fayne\AppData\Local\Temp\ipykernel_23232\909426180.py:23: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [ ]:
df = pd.read_csv("task-3-dataset.csv") # Считываем данные

разметка
+    121
-     89
Name: count, dtype: int64

In [70]:
review = df['отзывы'].loc[6]
df

,отзывы,разметка
0,Оболочка после чистого андроида тоже очень нек...,-
1,"Нормальный телефон, очень красивая задняя панель",+
2,Деньги на ветер .,-
3,ну так себе,-
4,Ценник вполне адекватный для такой мощной нови...,+
...,...,...
205,"Короче, можно брать. Но есть моменты, которые ...",+
206,очень крутой дизайн,+
207,"Звук ужасный, собеседника очень плохо слышно, ...",-
208,10 раз подумайте прежде чем покупать. Весьма с...,-


In [148]:
import pymorphy3
morph = pymorphy3.MorphAnalyzer()
# Функция загрузки стопслов
def downloads_():
    import nltk
    nltk.download('stopwords')
    from nltk.corpus import stopwords
# Функция обработки текта
def foo(review, morph):
    # Обработка текста отзыва. Оставляем только буквы, приводим к нижнему регистру
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^а-яА-Я]', ' ', review)
    review = review.lower()
    # Отделяем слова
    review = review.split()
    # Избавляемся от стоп-слов(предлоги,союзы, частицы, не несущие семантической нагрузки)
    review = [word for word in review if not word in set(stopwords.words('russian'))]
    # Лемматизируем(для русского языка в явном виде нет, но пока тк)
    # В явном виде лемматизации нет для русского языка,  SNOWBALL STEMMER как вариант
    lemmatized_words = [morph.normal_forms(word)[0] for word in review]
    lemmatized_text = ' '.join(lemmatized_words)
    return lemmatized_text

In [186]:
# Разделим данные, пока нет тестовых
from sklearn.model_selection import train_test_split
dataset_train, dataset_test, train_data_label, test_data_label = train_test_split(df['отзывы'], df['разметка'], test_size=0.15, random_state=37)


In [187]:
# Сформируем тестовый и трейновый словари
corpus_train = []
corpus_test  = []

downloads_()
for i in range(dataset_train.shape[0]):
    review = dataset_train.iloc[i]
    review = foo(review, morph)
    corpus_train.append(review)

for j in range(dataset_test.shape[0]):
    review = dataset_test.iloc[j]
    review = foo(review, morph)
    corpus_test.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fayne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [188]:
# Векторизуем с помощью TF-IDF 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(ngram_range=(1, 3))

tfidf_vec_train = tfidf_vec.fit_transform(corpus_train)
tfidf_vec_test = tfidf_vec.transform(corpus_test)


In [189]:
# Обучаем
from sklearn.svm import LinearSVC

linear_svc = LinearSVC(C=0.5, random_state=42)
linear_svc.fit(tfidf_vec_train, train_data_label)

predict = linear_svc.predict(tfidf_vec_test)

In [191]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("Classification Report: \n", classification_report(test_data_label, predict,target_names=['Negative','Positive']))
print("Confusion Matrix: \n", confusion_matrix(test_data_label, predict))
print("Accuracy: \n", accuracy_score(test_data_label, predict))

Classification Report: 
               precision    recall  f1-score   support

    Negative       0.76      0.84      0.80        19
    Positive       0.73      0.62      0.67        13

    accuracy                           0.75        32
   macro avg       0.74      0.73      0.73        32
weighted avg       0.75      0.75      0.75        32

Confusion Matrix: 
 [[16  3]
 [ 5  8]]
Accuracy: 
 0.75


In [192]:
# Векторизуем с помощью другого векторизатора   
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer(ngram_range=(1, 3), binary=False)
count_vec_train = count_vec.fit_transform(corpus_train)
count_vec_test = count_vec.transform(corpus_test)

linear_svc_count = LinearSVC(C=0.5, random_state=42, max_iter=5000)
linear_svc_count.fit(count_vec_train, train_data_label)
predict_count = linear_svc_count.predict(count_vec_test)

print("Classification Report: \n", classification_report(test_data_label, predict_count,target_names=['Negative','Positive']))
print("Confusion Matrix: \n", confusion_matrix(test_data_label, predict_count))
print("Accuracy: \n", accuracy_score(test_data_label, predict_count))

Classification Report: 
               precision    recall  f1-score   support

    Negative       0.71      0.79      0.75        19
    Positive       0.64      0.54      0.58        13

    accuracy                           0.69        32
   macro avg       0.68      0.66      0.67        32
weighted avg       0.68      0.69      0.68        32

Confusion Matrix: 
 [[15  4]
 [ 6  7]]
Accuracy: 
 0.6875


In [193]:
# И снова другой векторизатор
ind_vec = CountVectorizer(ngram_range=(1, 3), binary=True)
ind_vec_train = ind_vec.fit_transform(corpus_train)
ind_vec_test = ind_vec.transform(corpus_test)

linear_svc_ind = LinearSVC(C=0.5, random_state=42)
linear_svc_ind.fit(ind_vec_train, train_data_label)
predict_ind = linear_svc_ind.predict(ind_vec_test)

print("Classification Report: \n", classification_report(test_data_label, predict_ind,target_names=['Negative','Positive']))
print("Confusion Matrix: \n", confusion_matrix(test_data_label, predict_ind))
print("Accuracy: \n", accuracy_score(test_data_label, predict_ind))

Classification Report: 
               precision    recall  f1-score   support

    Negative       0.73      0.84      0.78        19
    Positive       0.70      0.54      0.61        13

    accuracy                           0.72        32
   macro avg       0.71      0.69      0.69        32
weighted avg       0.72      0.72      0.71        32

Confusion Matrix: 
 [[16  3]
 [ 6  7]]
Accuracy: 
 0.71875


In [194]:
# TF_IDF дал лучший результат, используем его, добаим наивный байесовский классификатор
tfidf_vec_NB = TfidfVectorizer(ngram_range=(1, 1))
tfidf_vec_train_NB = tfidf_vec_NB.fit_transform(corpus_train)

tfidf_vec_test_NB = tfidf_vec_NB.transform(corpus_test)

print(tfidf_vec_train_NB.toarray().shape, tfidf_vec_test_NB.toarray().shape)

(178, 770) (32, 770)


In [195]:
from sklearn.feature_selection import SelectKBest, chi2

ch2 = SelectKBest(chi2, k=50000)
tfidf_vec_train_NB = ch2.fit_transform(tfidf_vec_train_NB, train_data_label)
tfidf_vec_test_NB  = ch2.transform(tfidf_vec_test_NB)

c:\Users\fayne\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:776: UserWarning: k=50000 is greater than n_features=770. All the features will be returned.
  warnings.warn(


In [203]:
feature_names = tfidf_vec_NB.get_feature_names_out()
feature_names = [feature_names[i] for i
                         in ch2.get_support(indices=True)]
feature_names = np.asarray(feature_names)

from sklearn.naive_bayes import MultinomialNB
multi_clf = MultinomialNB()
multi_clf.fit(tfidf_vec_train_NB, train_data_label)
predict_NB = multi_clf.predict(tfidf_vec_test_NB)

print("Classification Report: \n", classification_report(test_data_label, predict_NB,target_names=['Negative','Positive']))
print("Confusion Matrix: \n", confusion_matrix(test_data_label, predict_NB))
print("Accuracy: \n", accuracy_score(test_data_label, predict_NB))

Classification Report: 
               precision    recall  f1-score   support

    Negative       0.72      0.95      0.82        19
    Positive       0.86      0.46      0.60        13

    accuracy                           0.75        32
   macro avg       0.79      0.70      0.71        32
weighted avg       0.78      0.75      0.73        32

Confusion Matrix: 
 [[18  1]
 [ 7  6]]
Accuracy: 
 0.75


In [204]:
count_vec_NB = CountVectorizer(ngram_range=(1, 3), binary=False)
count_vec_train_NB = count_vec_NB.fit_transform(corpus_train)
count_vec_test_NB = count_vec_NB.transform(corpus_test)

multi_clf_count = MultinomialNB()
multi_clf_count.fit(count_vec_train_NB, train_data_label)
predict_NB_count = multi_clf_count.predict(count_vec_test_NB)

print("Classification Report: \n", classification_report(test_data_label, predict_NB_count,target_names=['Negative','Positive']))
print("Confusion Matrix: \n", confusion_matrix(test_data_label, predict_NB_count))
print("Accuracy: \n", accuracy_score(test_data_label, predict_NB_count))

Classification Report: 
               precision    recall  f1-score   support

    Negative       0.77      0.89      0.83        19
    Positive       0.80      0.62      0.70        13

    accuracy                           0.78        32
   macro avg       0.79      0.76      0.76        32
weighted avg       0.78      0.78      0.77        32

Confusion Matrix: 
 [[17  2]
 [ 5  8]]
Accuracy: 
 0.78125


In [205]:
# Попробуем теперь LSTM НАХУЙ
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Masking, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

ModuleNotFoundError: No module named 'tensorflow'